In [1]:
import json
from kafka import KafkaConsumer
import pandas as pd
import re

In [2]:
consumer1  =KafkaConsumer(
    'item',
     bootstrap_servers=['localhost:9092'],
     auto_offset_reset='latest',
     enable_auto_commit=True,
     group_id='my-group')

consumer2  =KafkaConsumer(
    'voucher',
     bootstrap_servers=['localhost:9092'],
     auto_offset_reset='latest',
     enable_auto_commit=True,
     group_id='my-group1')

In [3]:
def fetch_data():
    data_item = []
    no = 0
    for message in consumer1:
        message = message.value
    #     print(message)
        data_item.append(eval(message.decode("utf-8") .replace('null','None').replace('false','False').replace('true','True')))
        no+=1
        if no==4:
            break

    data_voucher = []
    no = 0
    for message in consumer2:
        message = message.value
    #     print(message)
        data_voucher.append(eval(message.decode("utf-8") .replace('null','None').replace('false','False').replace('true','True')))
        no+=1
        if no==4:
            break


    item = pd.DataFrame.from_records(data_item)
    voucher = pd.DataFrame.from_records(data_voucher)
    df = pd.merge(item, voucher, how='left',on=['itemid','shopid','scraped_start_date'])
    return df

In [4]:
# For reset topic
# for message in consumer1:
#     message = message.value
# for message in consumer2:
#     message = message.value

In [5]:
import numpy as np

In [6]:
def calculate_finalprice(df):
    final_price= []
    for j in range(len(df)):
        print('price_before',df.iloc[j]['price'])
        min_price = 0
        discount = 0
        print(df.iloc[j]['shop_vouchers'])
        if type(df.iloc[j]['shop_vouchers']) != float:
            for i in eval(df.iloc[j]['shop_vouchers']):
                if i['min_spend']//100000 < df.iloc[j]['price'] and i['min_spend']//100000 > min_price  :
                    min_price = i['min_spend']//100000
                    discount = i['discount_value']//100000
                    print(discount)
        final_price.append(df.iloc[j]['price']-discount)
        print('price_after',df.iloc[j]['price']-discount)
    df['final_price'] = final_price
    return df

In [7]:
df = fetch_data()
main_df = calculate_finalprice(df)

price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 3599.0
nan
price_after 3599.0


In [8]:
import plotly.graph_objects as go
import numpy as np

def cal_diff(temp):
    temp = temp[['price','itemid']]

    price_diff = []
    for i in temp['itemid'].unique().tolist():
        if i != 15837488361:
            price_diff.append(temp[temp['itemid'] == i]['price'].values[0] - temp[temp['itemid'] == 15837488361]['price'].values[0])
        else:
            price_diff.append(0)
        print(i)
    temp['price_diff'] = price_diff
    temp["Color"] = np.where(temp["price_diff"]<0, 'red', 'green')
    
    t = pd.DataFrame()
    sort_list = [15837488361,12322236008,4419865091,7955667678]
    for i in sort_list:
        t = t.append(temp[temp['itemid']==i])
    return temp

In [9]:
import requests
def line_notify(df):
    
    for i in range(len(df)):
        if type(df.iloc[i]['flash_sale'] ) != float and df.iloc[i]['flash_sale']!= None:
            print('Flash sale!!')
            print(df.iloc[i]['flash_sale'])
            msg = ''' Flash sale!! 
            Price : {0}
            Url : {1}
            '''.format(df.iloc[i]['price'], 'https://shopee.co.th/a'+re.search('-i.+[0-9]', df.iloc[i]['url']).group(0)  )
            
            token ='aWpVFgC2m9gl7o8HZN2huwrcKfIEd977OkQLKvJyJGf'
            url = 'https://notify-api.line.me/api/notify'
            
            headers = {'content-type':'application/x-www-form-urlencoded','Authorization':'Bearer '+token}
            
            r = requests.post(url, headers=headers, data = {'message':msg})
            
            
            

            

In [10]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
# code and plot setup
# settings
pd.options.plotting.backend = "plotly"
countdown = 20
#global df

# sample dataframe of a wide format
np.random.seed(4); cols = list('abc')
X = np.random.randn(50,len(cols))  
df=pd.DataFrame(X, columns=cols)
df.iloc[0]=0;

# plotly figure
fig = df.plot(template = 'plotly_dark')

app = JupyterDash(__name__)
app.layout = html.Div([
    html.Div(children=[
                                    html.H1("Price Trend"),
                                            dcc.Interval(
                                            id='interval-component',
                                            interval=1*1000, # in milliseconds
                                            n_intervals=0
                                        ),
                                    dcc.Graph(id='graph'),
        
                                html.H1("Price comparison"),
                                            dcc.Interval(
                                            id='interval-component',
                                            interval=1*1000, # in milliseconds
                                            n_intervals=0
                                        ),
                                    dcc.Graph(id='graph2'),]),
])

# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    Output('graph2', 'figure'),
    [Input('interval-component', "n_intervals")]
)
def streamFig(value):
    
    global main_df
    
    df = fetch_data()
    df = calculate_finalprice(df)
    main_df = main_df.append(df)
    main_df.reset_index(drop=True,inplace=True)
    fig = px.line(main_df, 
        x="scraped_start_date", y="final_price", color='itemid')
    
    temp = cal_diff(df)
    fig2 = go.Figure()
    fig2.add_trace(go.Bar(x=temp['itemid'].astype(str), y=temp['price_diff'],
                    base=0,
                    marker_color=temp['Color'],
                    name='revenue'
                    ))

    line_notify(df)
    return[(fig),(fig2)]



C:\Users\Best\miniconda3\envs\dad6005\lib\site-packages\ipykernel_launcher.py:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  
C:\Users\Best\miniconda3\envs\dad6005\lib\site-packages\ipykernel_launcher.py:7: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import sys


In [11]:
app.run_server()

Dash app running on http://127.0.0.1:8050/
price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 3599.0
nan
price_after 3599.0
15837488361
12322236008
4419865091
7955667578


C:\Users\Best\miniconda3\envs\dad6005\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Best\miniconda3\envs\dad6005\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 3599.0
nan
price_after 3599.0
15837488361
12322236008
4419865091
7955667578
price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 3599.0
nan
price_after 3599.0
15837488361
12322236008
4419865091
7955667578
price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 3599.0
nan
price_after 3599.0
15837488361
12322236008
4419865091
7955667578
price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 3599.0
nan
price_after 3599.0
15837488361
12322236008
4419865091
7955667578
price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0

price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 3690.0
nan
price_after 3690.0
price_before 3599.0
nan
price_after 3599.0
15837488361
12322236008
4419865091
7955667578
price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 3690.0
nan
price_after 3690.0
price_before 3599.0
nan
price_after 3599.0
15837488361
12322236008
4419865091
7955667578
price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 3690.0
nan
price_after 3690.0
price_before 3599.0
nan
price_after 3599.0
15837488361
12322236008
4419865091
7955667578
price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 3690.0
nan
price_after 3690.0
price_before 3599.0
nan
price_after 3599.0
15837488361
12322236008
4419865091
7955667578
price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 3690.0
nan
price_after 3690.0

price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 3590.0
nan
price_after 3590.0
price_before 3599.0
nan
price_after 3599.0
15837488361
12322236008
4419865091
7955667578
price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 3590.0
nan
price_after 3590.0
price_before 3599.0
nan
price_after 3599.0
15837488361
12322236008
4419865091
7955667578
price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 3590.0
nan
price_after 3590.0
price_before 3599.0
nan
price_after 3599.0
15837488361
12322236008
4419865091
7955667578
price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 3590.0
nan
price_after 3590.0
price_before 3599.0
nan
price_after 3599.0
15837488361
12322236008
4419865091
7955667578
price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 3590.0
nan
price_after 3590.0

...

price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 6999.0
nan
price_after 6999.0
price_before 3399.0
nan
price_after 3399.0
7955667578
15837488361
12322236008
4419865091
price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 6999.0
nan
price_after 6999.0
price_before 3399.0
nan
price_after 3399.0
7955667578
15837488361
12322236008
4419865091


C:\Users\Best\miniconda3\envs\dad6005\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Best\miniconda3\envs\dad6005\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 6999.0
nan
price_after 6999.0
price_before 3399.0
nan
price_after 3399.0
7955667578
15837488361
12322236008
4419865091
price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 6999.0
nan
price_after 6999.0
price_before 3399.0
nan
price_after 3399.0
7955667578
15837488361
12322236008
4419865091
price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 6999.0
nan
price_after 6999.0
price_before 3399.0
nan
price_after 3399.0
7955667578
15837488361
12322236008
4419865091
price_before 3299.0
nan
price_after 3299.0
price_before 3299.0
nan
price_after 3299.0
price_before 6999.0
nan
price_after 6999.0
price_before 3399.0
nan
price_after 3399.0
7955667578
15837488361
12322236008
4419865091
price_before 3299.0
nan
price_after 3299.0
price_before 6999.0
nan
price_after 6999.0
price_before 3399.0
nan
price_after 3399.0

price_before 3590.0
nan
price_after 3590.0
price_before 3299.0
nan
price_after 3299.0
price_before 3590.0
nan
price_after 3590.0
price_before 3299.0
nan
price_after 3299.0
price_before 3590.0
nan
price_after 3590.0
price_before 3299.0
nan
price_after 3299.0
price_before 3590.0
nan
price_after 3590.0
price_before 3299.0
nan
price_after 3299.0
price_before 3590.0
nan
price_after 3590.0
price_before 3299.0
nan
price_after 3299.0
price_before 3590.0
nan
price_after 3590.0
price_before 3299.0
nan
price_after 3299.0
price_before 3590.0
nan
price_after 3590.0
price_before 3299.0
nan
price_after 3299.0
price_before 3590.0
nan
price_after 3590.0
price_before 3299.0
nan
price_after 3299.0
price_before 3590.0
nan
price_after 3590.0
price_before 3299.0
nan
price_after 3299.0
price_before 3590.0
nan
price_after 3590.0
price_before 3299.0
nan
price_after 3299.0
price_before 3590.0
nan
price_after 3590.0
price_before 3299.0
nan
price_after 3299.0
price_before 3590.0
nan
price_after 3590.0
price_befor

price_before 3590.0
nan
price_after 3590.0
price_before 3299.0
nan
price_after 3299.0
price_before 3590.0
nan
price_after 3590.0
price_before 3299.0
nan
price_after 3299.0
price_before 3590.0
nan
price_after 3590.0
price_before 3299.0
nan
price_after 3299.0
price_before 3590.0
nan
price_after 3590.0
price_before 3299.0
nan
price_after 3299.0
price_before 3590.0
nan
price_after 3590.0
price_before 3299.0
nan
price_after 3299.0
price_before 3590.0
nan
price_after 3590.0
price_before 3299.0
nan
price_after 3299.0
price_before 3590.0
nan
price_after 3590.0
price_before 3299.0
nan
price_after 3299.0
price_before 3590.0
nan
price_after 3590.0
price_before 3299.0
nan
price_after 3299.0
price_before 3590.0
nan
price_after 3590.0
price_before 3299.0
nan
price_after 3299.0
price_before 3590.0
nan
price_after 3590.0
price_before 3299.0
nan
price_after 3299.0
price_before 3590.0
nan
price_after 3590.0
price_before 3299.0
nan
price_after 3299.0
price_before 3590.0
nan
price_after 3590.0
price_befor

price_before 3299.0
70
price_after 3229.0
price_before 3299.0
40
price_after 3259.0
price_before 3299.0
price_after 3299.0
price_before 3599.0
price_after 3599.0


,_id,scraped_datetime,data,model_name,modelid_x,scraped_start_date,is_indexable,stock,class_name,promotionid,...,price,price_before_discount,upcoming_flash_sale,shop_information,deep_discount,shipping,item_has_size_recommendation,shop_vouchers,modelid_y,final_price
0,shopee_price_alert_114598626619492182794471212...,1666163171,None,None,28388431479,2022-10-19T14:00:01,None,39,shopee_price_alert,117728342020096,...,3299.0,5990.0,None,"{'rating_bad': 942, 'userid': 28711413, 'follo...",None,None,None,"[{'has_expired': None, 'usage_limit': None, 's...",28388431479,3229.0
1,shopee_price_alert_942835723598493368438493168...,1666162915,None,4 lite KK39,87927560964,2022-10-19T14:00:01,None,184,shopee_price_alert,117550560120832,...,3299.0,6999.0,None,"{'rating_bad': 87, 'userid': 237182324, 'follo...",None,None,None,"[{'has_expired': None, 'usage_limit': None, 's...",87927560964,3259.0
2,shopee_price_alert_262304627045895519623476665...,1666162909,None,"Purifier 4 Lite,ชำระเต็มจำนวน",152375705934,2022-10-19T14:00:01,None,294,shopee_price_alert,678492679321729,...,3299.0,5990.0,None,"{'rating_bad': 1585, 'userid': 43989516, 'foll...",None,None,None,"[{'has_expired': None, 'usage_limit': None, 's...",152375705934,3299.0
3,shopee_price_alert_239402995584846973961375446...,1666163144,None,4 Lite ฺGB ศูนย์ไทย,161136045205,2022-10-19T14:00:01,None,1000,shopee_price_alert,667758524238459,...,3599.0,8990.0,None,"{'rating_bad': 601, 'userid': 53899059, 'follo...",None,None,None,[],161136045205,3599.0
